In [1]:
%matplotlib inline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.utils import resample
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
import time
import numpy as np
import pandas as pd
import json
import os
import matplotlib.pyplot as plt

In [2]:
SAVE_PATH_TRAIN = '/home/anhduc/result/train_set.json'
SAVE_PATH_TEST = '/home/anhduc/result/test_set.json'

DIR_PATH = os.path.dirname(os.path.realpath('/home/anhduc/Documents/data/text/'))
# load data
DATA_TRAIN_PATH = os.path.join(DIR_PATH, 'text/Train_Full/')
DATA_TEST_PATH = os.path.join(DIR_PATH, 'text/Test_Full/')
# Danh sach nhan
list_labels = os.listdir(path=DATA_TRAIN_PATH)
list_labels

['Phap luat',
 'Van hoa',
 'Suc khoe',
 'Vi tinh',
 'Khoa hoc',
 'Doi song',
 'Chinh tri Xa hoi',
 'The gioi',
 'The thao',
 'Kinh doanh']

In [3]:
def split_data(data, test_ratio):
    shuffled_id = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_ids = shuffled_id[:test_set_size]
    train_ids = shuffled_id[test_set_size:]
    return data.iloc[train_ids], data.iloc[test_ids]

In [4]:
df_train = pd.read_json(SAVE_PATH_TRAIN)
df_test = pd.read_json(SAVE_PATH_TEST)

In [5]:
data=pd.DataFrame()

In [6]:
data1=df_train[df_train.label==0]

In [7]:
for i in range(10):
    tmp=df_train[df_train.label==i]
    data=pd.concat([data, tmp.head(1000)], axis=0, ignore_index=True)

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
label      10000 non-null int64
content    10000 non-null object
dtypes: int64(1), object(1)
memory usage: 156.3+ KB


In [9]:
data.label.value_counts()

7    1000
6    1000
5    1000
4    1000
3    1000
2    1000
9    1000
1    1000
8    1000
0    1000
Name: label, dtype: int64

In [10]:
text_clf = Pipeline([ ('cv', CountVectorizer(stop_words=None, tokenizer=None, 
                                             min_df=5, max_df=0.8, 
                                             preprocessor=None, ngram_range=(1, 2), 
                                             max_features=50000, binary=True)), 
                     ('tfidf', TfidfTransformer()), 
                     ('clf', SVC(kernel='linear')) ])

In [11]:
text_clf.fit(data.content, data.label)

Pipeline(memory=None,
     steps=[('cv', CountVectorizer(analyzer='word', binary=True, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.8, max_features=50000, min_df=5,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip_a...r', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False))])

In [12]:
test=pd.DataFrame()

In [13]:
for i in range(10):
    tmp=df_test[df_test.label==i]
    test=pd.concat([test, tmp.head(1000)], axis=0, ignore_index=True)

In [14]:
label_pred = text_clf.predict(test.content)
np.mean(label_pred==test.label)

0.9038